# Ejercicio 1

A partir del archivo inegi.json localizado en '/files/lec06/' crear un numpy array que contenga la información del campo **OBS_VALUE**

**Sugerencia**

Lea la documentación de 

```python
numpy.append
```

In [1]:
import json
import numpy as np

In [2]:
with open('../../files/lec06/inegi.json') as f:
    datos = json.load(f)
observaciones = datos['Series'][0]['OBSERVATIONS'] 
observaciones

[{'TIME_PERIOD': '1993',
  'OBS_VALUE': '10165571.17700000000000000000',
  'OBS_EXCEPTION': None,
  'OBS_STATUS': '3',
  'OBS_SOURCE': '',
  'OBS_NOTE': '',
  'COBER_GEO': '0700'},
 {'TIME_PERIOD': '1994',
  'OBS_VALUE': '10667860.25000000000000000000',
  'OBS_EXCEPTION': None,
  'OBS_STATUS': '3',
  'OBS_SOURCE': '',
  'OBS_NOTE': '',
  'COBER_GEO': '0700'},
 {'TIME_PERIOD': '1995',
  'OBS_VALUE': '9996720.53800000000000000000',
  'OBS_EXCEPTION': None,
  'OBS_STATUS': '3',
  'OBS_SOURCE': '',
  'OBS_NOTE': '',
  'COBER_GEO': '0700'},
 {'TIME_PERIOD': '1996',
  'OBS_VALUE': '10673824.28100000000000000000',
  'OBS_EXCEPTION': None,
  'OBS_STATUS': '3',
  'OBS_SOURCE': '',
  'OBS_NOTE': '',
  'COBER_GEO': '0700'},
 {'TIME_PERIOD': '1997',
  'OBS_VALUE': '11404645.26200000000000000000',
  'OBS_EXCEPTION': None,
  'OBS_STATUS': '3',
  'OBS_SOURCE': '',
  'OBS_NOTE': '',
  'COBER_GEO': '0700'},
 {'TIME_PERIOD': '1998',
  'OBS_VALUE': '11993572.60900000000000000000',
  'OBS_EXCEPTION': None

In [5]:
obs_value = np.array([])
for elem in observaciones:
    obs = np.array(float(elem['OBS_VALUE']))
    obs_value = np.append(obs_value, obs)
print(obs_value)

[10165571.177 10667860.25   9996720.538 10673824.281 11404645.262
 11993572.609 12323822.137 12932921.342 12880621.885 12875489.668
 13061718.521 13573815.057 13887072.52  14511307.247 14843825.975
 15013577.681 14219998.379 14947794.696 15495333.601 16059723.651
 16277187.078 16733654.767 17283855.93  17786910.587 18163652.487
 18551620.026]


# Ejercicio 2

Utilizando el archivo csv *sp500.csv* que se encuentra en **intro-python/ejercicios/datos/**, programe una función que calcule el promedio móvil de la columna *Adj Close*, utilizando una ventana de tiempo de $n$ días.

**Ejemplo de cálculo**

Suponga que se tiene el vector precios ```p = [11,12,13,14,15,16,17]``` y que $n = 3$,  si $MA(t,p,n)$ denota el promedio móvil en el momento $t$ para la serie $p$ con una ventana de tiempo $n$, entonces:

* $MA(0,p,3) = \dfrac{11 + 12 + 13}{3}$

* $MA(1,p,3) = \dfrac{12 + 13 + 14}{3}$ 

* $MA(2,p,3) = \dfrac{13 + 14 + 15}{3}$

* $MA(3,p,3) = \dfrac{14 + 15 + 16}{3}$ 

* $MA(4,p,3) = \dfrac{15 + 16 + 17}{3}$ 

In [6]:
import numpy as np

def lee_datos(ruta = '../datos/sp500.csv'):
    '''
    Función para leer el csv
    '''
    with open(ruta, 'r') as f:
        datos = f.readlines()
    return datos

def obten_columna(datos, col = 5):
    '''
    Función para obtener los datos de la columna
    deseada
    
    ENTRADA
    datos: lista creada con la función lee_datos
    
    col: Entero que representa el índice de la columna de interés
    
    SALIDA
    arreglo de numpy
    '''
    
    #Aquí se almacenarán los datos de la columna de interés
    precios = np.array([])
    
    #Se inicia en 1 para omitir los encabezados
    for i in range(1, len(datos)):
        
        renglon = datos[i]
        valor = float(renglon.split(',')[col])
        precios = np.append(precios, valor)
    return precios    

def promedio_movil(precios, n = 3):
    '''
    Función para calcular el promedio móvil de una serie de tiempo
    
    ENTRADA
    precios: numpy array o lista con los valores de la serie de tiempo
    
    n: Entero positivo (ventana de tiempo)
    
    SALIDA
    numpy array con los promedios móviles
    '''
    
    #Para almacenar los promedios móviles
    prom_mov = np.array([])
    
    #número de observaciones
    n_obs = len(precios)
    
    #Si n = 3, entonces los índices son:
    #[0, 3) i = 0
    #[1, 4) i = 1
    #[2, 5) i = 2
    #[3, 6) i = 3
    #última iteración
    #[n_obs - 3, n_obs)
    for i in range(0, n_obs - 3 + 1): #el + 1 es porque se incluye el límite izquierdo [n_obs - 3
        promedio = np.mean(precios[i:i + n])
        prom_mov = np.append(prom_mov, promedio)
    return prom_mov

# Ejercicio 3

Utilizando el archivo *bono.csv*, programe una función para calcular el precio de un bono:

$$
Precio_{t_0} = \sum_{i = 1}^{n} \dfrac{cupon_i}{ \left(1 + r_i \right)^{\tau_i}} + \dfrac{Nominal}{ \left(1 + r_n \right)^{\tau_n}}
$$

En donde 

* $\tau_i = \dfrac{Fecha_i - Fecha_{valuacion}}{360}$

Considere como fecha de valuación , $t_0$, el día 29 de Noviembre de 2019 y un nominal de 100.

**Sugerencia**

Si se tienen dos objetos del tipo ```datetime.datetime```, $f1$ y $f2$, entonces su resta, $delta = f2 - f1$ es un objeto del tipo ```datetime.timedelta``` el cual tiene un atributo llamado ```days```.

In [16]:
from datetime import datetime, timedelta
def obten_columna_csv(datos, col = 2, bool_float = False):
    '''
    Función para obtener los datos de la columna
    deseada
    
    ENTRADA
    datos: lista creada con la función lee_datos
    
    col: Entero que representa el índice de la columna de interés
    
    bool_float: Booleano. True => se hace cast a float
    
    SALIDA
    arreglo de numpy
    '''
    
    #Aquí se almacenarán los datos de la columna de interés
    columna = np.array([])
    
    #Se inicia en 1 para omitir los encabezados
    for i in range(1, len(datos)):
        
        renglon = datos[i]
        valor = renglon.split(',')[col]
        valor = valor.replace('\n','')
        if bool_float:
            valor = float(valor)
        columna = np.append(columna, valor)
    return columna

In [17]:
datos = lee_datos('../datos/bono.csv') #Del ejercicio 2
fechas = obten_columna_csv(datos, 0)
cupones = obten_columna_csv(datos, 1, True)
tasas = obten_columna_csv(datos, 2, True)

In [18]:
def valua_bono(fechas, cupones, tasas, fecha_val = '2019/11/29', nominal = 100):
    
    precio = 0
    fecha_val = datetime.strptime(fecha_val, '%Y/%m/%d')
    
    for f, c, t in zip(fechas,cupones,tasas):
        
        #Convierte la fecha a datetime
        f = datetime.strptime(f, '%m/%d/%y')
        
        #Calcula tau
        delta = f - fecha_val
        tau = delta.days / 360.0
        
        precio = precio + c / ((1 + t)**tau)
    
    #Valor presente del nominal
    ultima_fecha = datetime.strptime(fechas[-1], '%m/%d/%y')
    delta = ultima_fecha - fecha_val
    tau = delta.days / 360.0
    
    precio = precio + nominal / ((1 + tasas[-1])**tau)
    
    return precio


In [19]:
precio = valua_bono(fechas, cupones, tasas)
precio

145.07748820956095